# MCP Authentication Flow - Illustrated

This notebook illustrates the end-to-end authentication flow for Model Context Protocol (MCP). The flow involves several key steps, including metadata discovery and parsing, client registration, user authentication, token generation, and access validation.

A descripition of this flow is available in the [MCP Authentication Documentation](https://modelcontextprotocol.io/docs/tutorials/security/authorization).

This notebook illustrates the authentication flow with the demo server.

In [108]:
$env:McpServer = "http://localhost:5522/"

## Step 1 - Initial Handshake

The initial handshake begins when a client application attempts to access a protected resource on the MCP server. The server responds with 401 Unauthorized status code and provides the client with the MCP metadata URL in the `WWW-Authenticate` header.


In [109]:
curl -s -D - -X POST `
  -H "Content-Type: application/json" `
  -H "Accept: application/json, text/event-stream" `
  -d '{"jsonrpc":"2.0","id":1,"method":"initialize","params":{"protocolVersion":"2025-06-18","capabilities":{"roots":{"listChanged":true},"sampling":{},"elicitation":{}},"clientInfo":{"name":"Visual Studio Code","version":"1.105.1"}}}' `
 "$env:McpServer"

HTTP/1.1 401 Unauthorized
Content-Length: 0
Date: Wed, 22 Oct 2025 16:15:03 GMT
Server: Kestrel
WWW-Authenticate: Bearer realm="McpAuth", resource_metadata="http://localhost:5522/.well-known/oauth-protected-resource"



## Step 2 - Protected Resource Metadata Discovery

When receiving a 401 response in Step 1, the client extracts the MCP metadata URL from the `WWW-Authenticate` header. Then it sends a GET request to this URL to retrieve the Protected Resource Metadata (PRM) document, which contains information about the authentication server(s) that can be used to authenticate and obtain access tokens.

In [110]:
curl -s -X GET `
  -H "Accept: application/json" `
 "http://localhost:5522/.well-known/oauth-protected-resource"

{"resource":"http://localhost:5522","authorization_servers":["http://localhost:8080/realms/local"],"bearer_methods_supported":["header"],"scopes_supported":["mcp:tools"],"resource_documentation":"https://docs.example.com/api/math"}


## Step 3 - Authorization Server Discovery

The PRM document should include one or more authorization servers in the `authorization_servers` array. When there is more than one, the client can choose which one to use, but in this case there is only one available. Next, the client retrieves metadata for the authorization server. This could be either OpenID Connect (OIDC) discovery metadata or OAuth 2.0 Authorization Server Metadata, depending on what the authorization server supports. There's no clear indication in the PRM document of which type it is, so the client must try both methods.

According to the [OIDC specification], the client appends `/.well-known/openid-configuration` to the "issuer" URL to get the OIDC discovery document. The client sends a GET request to this URL and receives the OIDC metadata in response. Unfortunately, at this point all we have is the authorization server's URL, so we'll just try to use that as the issuer URL and hope for the best.

[OIDC specification]: https://openid.net/specs/openid-connect-discovery-1_0.html#ProviderConfig

In [111]:
$env:MCP_Auth_Server = "http://localhost:8080/realms/local"
$env:MCP_OIDC_Metadata = "$env:MCP_Auth_Server/.well-known/openid-configuration"

curl -s -X GET `
  -H "Accept: application/json" `
 "$env:MCP_OIDC_Metadata" | jq .

{
  "issuer": "http://localhost:8080/realms/local",
  "authorization_endpoint": "http://localhost:8080/realms/local/protocol/openid-connect/auth",
  "token_endpoint": "http://localhost:8080/realms/local/protocol/openid-connect/token",
  "introspection_endpoint": "http://localhost:8080/realms/local/protocol/openid-connect/token/introspect",
  "userinfo_endpoint": "http://localhost:8080/realms/local/protocol/openid-connect/userinfo",
  "end_session_endpoint": "http://localhost:8080/realms/local/protocol/openid-connect/logout",
  "frontchannel_logout_session_supported": true,
  "frontchannel_logout_supported": true,
  "jwks_uri": "http://localhost:8080/realms/local/protocol/openid-connect/certs",
  "check_session_iframe": "http://localhost:8080/realms/local/protocol/openid-connect/login-status-iframe.html",
  "grant_types_supported": [
    "authorization_code",
    "client_credentials",
    "implicit",
    "password",
    "refresh_token",
    "urn:ietf:params:oauth:grant-type:device_code"

What we are looking for in this metadata is the authorization endpoint, token endpoint, and supported scopes, which will be used in the subsequent steps of the authentication flow. And the scopes are clearly indicated in the `scopes_supported` field of the OIDC metadata.

But we don't see the authorization endpoint or token endpoint in the OIDC metadata. In this case, the best approach, and the one that happens to work with GitHub, is to assume the standard OAuth 2.0 endpoint paths:
- Authorization Endpoint: `{authorization_server_url}/protocol/openid-connect/auth`
- Token Endpoint: `{authorization_server_url}/protocol/openid-connect/token`

In [112]:
$env:Scopes =  "openid%20mcp:tools"

$env:AuthorizeEndpoint = "$env:MCP_Auth_Server/protocol/openid-connect/auth"
$env:TokenEndpoint = "$env:MCP_Auth_Server/protocol/openid-connect/token"

## Step 4 - Client Registration

When the authorization server supports dynamic client registration, the authorization server metadata will include a `registration_endpoint` field. The client can use this endpoint to register itself with the authorization server by sending a POST request with its details, such as redirect URIs and client name. The server responds with a client ID and client secret, which the client will use in subsequent authentication requests.

### Manual Client Registration

TBD

In [113]:
Add-Type -AssemblyName System.Web

$env:CallbackUrl = "http://127.0.0.1/"
$env:CallbackUrlEncoded = [System.Web.HttpUtility]::UrlEncode($env:CallbackUrl)


Store the client ID and client secret securely, as they will be needed in the next steps of the authentication flow. For this illustration, the client ID and client secret are stored in a .env file which is not checked into source control.


In [114]:
# Load .env file into the $dotenv hashtable
$dotenv = & "./Load-DotEnv.ps1"

## Step 5 - User Authorization

In this step, the client issues a GET request to the authorization endpoint obtained in Step 3. This request includes parameters such as the client ID, redirect URI, requested scopes, and response type (typically "code" for the authorization code flow). The user is then redirected to the authorization server's login page to authenticate and authorize the client application.

In [115]:
$scopes = [System.Web.HttpUtility]::UrlEncode($env:CallbackUrl)

$url = "$($env:AuthorizeEndpoint)?response_type=code&client_id=$($dotenv['ClientId'])&redirect_uri=$env:CallbackUrlEncoded&scope=$env:Scopes"
echo $url

http://localhost:8080/realms/local/protocol/openid-connect/auth?response_type=code&client_id=ee6ed1c1-f470-4f90-ac9b-bae05ae3bfa8&redirect_uri=http%3a%2f%2f127.0.0.1%2f&scope=openid%20mcp:tools


Get the code from the redirect URL after user authorization.

In [116]:
$env:authCode="4fa7c1de-508c-afda-6f68-58395ad70fd9.56251b8e-1fb4-ae77-61c7-9ceebefd680b.ee6ed1c1-f470-4f90-ac9b-bae05ae3bfa8"

Once the client has received the authorization code, it can exchange it for an access token using the token endpoint obtained in Step 3.
The client sends a POST request to the token endpoint with parameters such as the client ID, authorization code, redirect URI, and grant type (typically "authorization_code"). The server responds with an access token (and optionally a refresh token) that the client can use to access protected resources on behalf of the user.

The response of this request will include an access_token, which you can use to authenticate subsequent requests to the demo server.

In [117]:
$body = "grant_type=authorization_code" +
        "&client_id=$($dotenv['ClientId'])" +
        "&code=$($env:authCode)" +
        "&redirect_uri=$($env:CallbackUrl)"

$response = curl -s -X POST `
  -H "Content-Type: application/x-www-form-urlencoded" `
  -H "Accept: application/json" `
  -d "$body" `
  "$env:TokenEndpoint"


$env:accessToken = ($response | ConvertFrom-Json).access_token

## Step 6 - Making Authenticated Requests

Here's an example of how to use the access token to make an authenticated request:

In [119]:
curl -s -X POST `
  -H "Authorization: Bearer $env:accessToken" `
  -H "Content-Type: application/json" `
  -H "Accept: application/json, text/event-stream" `
  -d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "clientInfo": {
            "name": "Polyglot Notebook",
            "version": "0.1.0"
        },
        "capabilities": {},
        "protocolVersion": "2025-06-18"
    }
}' `
 "$env:McpServer"

event: message
data: {"result":{"protocolVersion":"2025-06-18","capabilities":{"logging":{},"tools":{"listChanged":true}},"serverInfo":{"name":"SampleAspNetCoreMcp.ApiService","version":"1.0.0.0"}},"id":1,"jsonrpc":"2.0"}

